<a href="https://colab.research.google.com/github/guilherme-mantovani/FakeNewsDetector/blob/main/FakeNewsDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalando bibliotecas

In [11]:
#Para ler vídeo do youtube
!pip install -q -U pytube

#Gemini
!pip install -q -U google-generativeai

#extração de áudio
!pip install -q -U moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00


## Importando bibliotecas principais e configurando credenciais

Para salvar as credenciais do google, está sendo utilizada a função Secrets, que fica aqui à esquerda ⬅

Crie uma secret com nome GOOGLE_API_KEY com a key de acesso ao gemini, e outra GOOGLE_CX com o id da busca customizada gerada conforme a explicação abaixo:

**# Importante**

**Para este projeto, é altamente recomendável ativar na mesma chave utilizada para o Gemini o produto Custom search API**

Segui o tutorial descrito em: https://drlee.io/build-your-own-google-create-a-custom-search-engine-with-trusted-sources-c1c113e845cc

Para facilitar, se esta chave não estiver cadastrada, o programa ainda irá funcionar, mas não há garantias de fontes reais atuais de informação, com o link de onde a notícia pode ser encontrada, o que deixa a utilidade bem inferior e menos impressionante

In [19]:
import google.generativeai as genai
import json
import os

from google.colab import userdata
#armazenando em uma variável pois será reutilizado para API de busca
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GOOGLE_CX = userdata.get('GOOGLE_CX')
genai.configure(api_key=GOOGLE_API_KEY)

Aqui são definidas algumas funções que serão utilizadas posteriormente, para extrair frames do vídeo (1 por segundo), e converter em array de imagens

In [13]:
import cv2
import shutil

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "/content/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

Esta função exclui os dados da base do Gemini **gerados nesta aplicação**. Importante pra não lotar o armazenamento. Se não rodar no final, os arquivos são apagados automaticamente 2 dias após (segundo a documentação)

In [14]:
#variável que armazena o áudio do vídeo, declarando agora para poder excluir quando necessário
response_mp3 = ""

def delete_files():
  print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
  for file in uploaded_files:
    genai.delete_file(file.response.name)
    print(f'Deleted {file.file_path} at URI {file.response.uri}')
  if(response_mp3 != ""):
    genai.delete_file(response_mp3)
    print(f'Deleted {response_mp3}')
  print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")

# A partir do segundo teste

**Se estiver realizando o segundo teste, pode começar a executar o código daqui!**


Nesta seção, o sistema cria dois botões para o usuário escolher a forma de importação do vídeo. Por se tratar de uma imersão Alura+Google, foi desenvolvido usando o Youtube ou upload de vídeo, mas poderiam se criar outras formas posteriormente como outras redes sociais ou sites, além de outras fontes como imagens.

# Importante!

Por limitações dos serviços, APIs, processamento e tempo disponível de desenvolvimento, escolha vídeos curtos para testes (menos de 1 minuto para youtube, menos de 30 segundos para upload, segundo os poucos testes feitos por mim).

In [51]:
# @title
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display
from google.colab import files
from IPython.display import clear_output

display(HTML('<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"/>'))

#Descrição dos botões (pra não precisar repetir o texto no if)
d_b_yt = "Link do vídeo (youtube)"
d_b_up = "Upload de vídeo"


button_yt = widgets.Button(description=d_b_yt, icon='youtube', button_style='danger')
output = widgets.Output()
button_up = widgets.Button(description=d_b_up, icon='upload', button_style='info')

tipo = '';

def on_button_clicked(b):
  global tipo
  # Display the message within the output widget.
  if(b.description == d_b_yt):
    tipo = 'yt'
  else:
    tipo = 'up'
  print("Tipo escolhido: "+b.description)

button_yt.on_click(on_button_clicked)
button_up.on_click(on_button_clicked)

display(button_yt, output)
display(button_up, output)

Button(button_style='danger', description='Link do vídeo (youtube)', icon='youtube', style=ButtonStyle())

Output()

Button(button_style='info', description='Upload de vídeo', icon='upload', style=ButtonStyle())

Output()

Tipo escolhido: Upload de vídeo


Nesta seção, será feito o upload do vídeo ou download (se for do Youtube)

In [52]:
if(tipo == ''):
  print("Por favor clique em algum botão na seção anterior!")
elif(tipo == 'yt'):
  video = input("Digite a url do vídeo do youtube")
  from pytube import YouTube
  yt = YouTube(video)
  video = yt.streams.first().download()
else:
  uploaded = files.upload()
  #não precisaria usar o for aqui, mas não sei programar em python e isso funciona :)
  for fn in uploaded.keys():
    video = fn

Saving Vídeo do WhatsApp de 2024-05-09 à(s) 18.31.11_7268a542.mp4 to Vídeo do WhatsApp de 2024-05-09 à(s) 18.31.11_7268a542 (1).mp4


Aqui o sistema envia para o Google as imagens obtidas nos frames. Como vídeos muito grandes estavam falhando, são selecionados apenas uma quantia de frames pequena para enviar.

*@todo Dar um jeito de funcionar para mais frames*

In [53]:
class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

#Chama a função de extração de frames
extract_frame_from_video(video)

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
# Only upload a {max_frames} second slice of files to reduce upload time.
# Change full_video to True to upload the whole video.
full_video = False
max_frames = 20

uploaded_files = []
print(f'Uploading {len(files_to_upload) if full_video else max_frames} files. This might take a bit...')

for file in files_to_upload if full_video else files_to_upload[0:max_frames]:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

Extracting Vídeo do WhatsApp de 2024-05-09 à(s) 18.31.11_7268a542 (1).mp4 at 1 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 20 frames
Uploading 20 files. This might take a bit...
Uploading: /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:00.jpg...
Uploading: /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:01.jpg...
Uploading: /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:02.jpg...
Uploading: /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:03.jpg...
Uploading: /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:04.jpg...
Uploading: /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:05.jpg...
Uploading: /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:06.jpg...
Uploading: /content/f

Aqui começamos a utilizar a IA!!!

Primeiramente fazemos uma descrição do vídeo, baseado nos frames enviados (apenas as imagens) e armazenamos a resposta

In [54]:
# Create the prompt.
prompt = "Descreva o vídeo no idioma nativo do vídeo"

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files)
response_video = model.generate_content(request,
                                  request_options={"timeout": 600})

Agora extraímos o áudio do vídeo e armazenamos num mp3

In [55]:
from moviepy.editor import VideoFileClip

# Define the input video file and output audio file
mp3_file = "audio.mp3"

# Load the video clip
#video_clip = VideoFileClip(yt)
video_clip = VideoFileClip(video)

# Extract the audio from the video clip
audio_clip = video_clip.audio

# Write the audio to a separate file
audio_clip.write_audiofile(mp3_file)

# Close the video and audio clips
audio_clip.close()
video_clip.close()

print("Audio extraction successful!")
response_mp3 = genai.upload_file(mp3_file)

MoviePy - Writing audio in audio.mp3


MoviePy - Done.
Audio extraction successful!


Utilizando o MP3 gerado, e a técnica de FEW-SHOTS ensinada nas aulas,  realizamos a transcrição do áudio seguindo o padrão desejado

In [56]:
prompt = {"exemplos":[{
      "tempo":"00:00 - 00:10",
      "autor":"mulher 1",
      "audio":"Estou muito contente hoje"
    },
    {
      "tempo":"00:00 - 00:10",
      "autor":"som ambiente",
      "audio":"*barulho de passarinhos*"
    },
    {
      "tempo":"00:11 - 00:20",
      "autor":"Homem 1",
      "audio":"Que bom, também estou muito contente"
    }
          ]}
exemplos = json.dumps(prompt)
#como eu quero uma transcrição precisa, sem criatividade, vou deixar com temperatura 0
generation_config = {
  "temperature": 0,
  "response_mime_type": "application/json",
}
response_audio = model.generate_content(["Transcreva o áudio em json conforme o exemplo: "+exemplos, response_mp3],
                                  request_options={"timeout": 600}, generation_config=generation_config)

Utilizando novamente a técnica de FEW-SHOTS, vamos agora juntar áudio e vídeo para classificar o texto, contextualizar, definir um assunto, um level de periculosidade, já estimar uma probabilidade de ser uma notícia falsa e elencar sites confiáveis para buscar a informação.

Muitos destes dados poderão ser atualizados quando for realizada uma busca no Google, se o Custom Search foi criado.

Um sistema ou rede social poderia encerrar neste momento os dados para moderar os vídeos com alta probabilidade de serem falsos

In [57]:
model_simples = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")
exemplos = [{
    "assunto":"Mulher explicando como fazer um bolo de chocolate",
    "periculosidade_divulgacao":"1/10",
    "justificativa_periculosidade":"Não há riscos envolvidos em se acreditar numa receita de bolo de chocolate",
    "prompt_google":"Receita bolo de chocolate",
    "probabilidade_fake_news":"1%",
    "justificativa_probabilidade_fake_news":"Este vídeo aparentemente não é uma fake new, pois além de se tratar de um assunto costumeiro e sem relevância, é totalmente plausível que uma mulher faça bolo de chocolate da forma como apresentado",
    "recomendavel_buscar_fontes":False,
    "sugestao_sites_busca":["https://tudogostoso.com.br", "https://www.receitasnestle.com.br/"]
},
{
    "assunto":"Homem narrando invasão alienígena",
    "periculosidade_divulgacao":"5/10",
    "justificativa_periculosidade":"Há riscos em se acreditar nisso, pois poderia causar pânico desnecessariamente, embora seja provável que a maior parte da população não acredite. Não compartilhe este link sem ter certeza de fontes confiáveis",
    "prompt_google":"Invasão alienígena - notícia",
    "probabilidade_fake_news":"99%",
    "justificativa_probabilidade_fake_news":"Este vídeo aparentemente é uma fake new, pois é extremamente improvável que uma invasão alienígena esteja ocorrendo",
    "recomendavel_buscar_fontes":True,
    "sugestao_sites_busca":["https://g1.globo.com/", "https://noticias.uol.com.br/", "https://www.cnnbrasil.com.br/"]
},
{
    "assunto":"Homem descrevendo remédio milagroso para curar a Covid",
    "periculosidade_divulgacao":"9/10",
    "justificativa_periculosidade":"Há um extremo risco em se acreditar nisso sem fontes confiáveis, pois impede que portadores da doença se tratem corretamente. Não compartilhe este link sem ter certeza de fontes confiáveis",
    "prompt_google":"Covid tratamento e recomendações",
    "probabilidade_fake_news":"95%",
    "justificativa_probabilidade_fake_news":"Este vídeo provavelmente é uma fake new, pois não há registros ou estudos comprovados deste remédio na cura da Covid",
    "recomendavel_buscar_fontes":True,
    "sugestao_sites_busca":["https://covid.saude.gov.br/", "https://www.gov.br/saude/pt-br/assuntos/covid-19", "https://www.who.int/emergencies/diseases/novel-coronavirus-2019"]
}
]
#convert o json em texto
exemplos = json.dumps(exemplos)

#Escolhi a temperatura de 0.3 para os dados serem ligeiramente criativos, mas evitando alucinações
generation_config = {
  "temperature": 0.3,
  "top_p": 0.95,
  "top_k": 0,
  "response_mime_type": "application/json",
}
#prompt = "Dado o vídeo "+yt+" com a seguinte descrição das imagens: "+response_video.text+"e a seguinte transcrição do áudio: "+response_audio.text+ ",  responda conforme o exemplo: "+exemplos
prompt = "Dado o vídeo "+video+" com a seguinte descrição das imagens: "+response_video.text+"e a seguinte transcrição do áudio: "+response_audio.text+ ",  responda conforme o exemplo: "+exemplos
response_json = model.generate_content(prompt, generation_config=generation_config)

#convert o texto em json
json_test = json.loads(response_json.text)

Hora de buscar fontes recentes sobre a notícia ou tópico do vídeo. A inteligência artifical fica assim mais precisa pois tem fontes em tempo real.

É interessante ressaltar que com o Custom Search é possível criar Whitelist de fontes confiáveis ou Blacklists de fontes não confiáveis. Mas cuidado para isso não tornar o software enviesado em suas respostas! Quanto mais fontes confiáveis melhor.

# Importante

A API de busca do Google (Custom Search) permite apenas 100 buscas por dia no nível gratuíto. É mais do que o suficiente para testes, mas não é possível utilizar como um produto desta maneira.

In [58]:
buscar_google = True
try:
  api_key = GOOGLE_API_KEY
  cx = GOOGLE_CX
except NameError:
  buscar_google = False

if buscar_google:
  import requests

  query = json_test[0]['prompt_google']

  url = f"https://www.googleapis.com/customsearch/v1?key={api_key}&cx={cx}&q={query}"
  response = requests.get(url)
  data = json.loads(response.text)

  # Check for errors or empty search results
  if 'error' in data:
      print("Error:", data['error']['message'])
  elif 'items' not in data:
      print("No search results found.")

Aqui tabulamos e reduzimos os resultados vindos da API do Google para reduzir o número de tokens gerados para o Gemini

In [59]:
if buscar_google:
  # Extract search results
  search_results = data['items']
  results_google = []

  for result in search_results:
      title = result['title']
      link = result['link']
      snippet = result['snippet']
      results_google.append({'Title': title, 'Link': link, 'Snippet': snippet})

  results_google_json = json.dumps(results_google)

Hora de gerar uma conclusão sobre os dados, usando o json e a busca realizada no Google. Exibimos a conclusão para o usuário e as recomendações.

In [61]:
texto_busca_google = " e os resultados do Google sobre o assunto: "+results_google_json if buscar_google else ""
conclusao_request = "Dado o seguinte json: "+response_json.text+texto_busca_google+", que especifica a probabilidade de um vídeo enviado ser uma fake new. Caso sejam encontradas fontes confiáveis nos resultados do google, altere as informações do json para condizer com o encontrado antes de informar ao usuário. Redija um texto para o usuário citando todos os dados do json atualizado, links para se aprofundar no assunto - incluindo os links encontrados do google, recomendações sobre pesquisas e se deve compartilhar ou não o vídeo."
response_conclusao = model.generate_content(conclusao_request)
print(response_conclusao.text)

## Análise do vídeo: Homem falando sobre os perigos da vacina

**Periculosidade da Divulgação:** 9/10 -  Compartilhar este vídeo sem fontes confiáveis é extremamente arriscado, pois pode dissuadir pessoas de se vacinarem e se protegerem de doenças graves. 

**Probabilidade de Fake News:** 99% -  As vacinas são amplamente estudadas e comprovadas como seguras e eficazes por órgãos de saúde em todo o mundo. Portanto, este vídeo provavelmente contém informações falsas ou distorcidas.

**Recomendação:** **Não compartilhe este vídeo**. 

**Fontes Confiáveis para se Aproximar do Assunto:**

* **Governo Federal - Ministério da Saúde:** https://www.gov.br/saude/pt-br
* **Organização Mundial da Saúde (OMS):** https://www.who.int/en/
* **Centro de Controle e Prevenção de Doenças (CDC - EUA):** https://www.cdc.gov/
* **CNN Brasil - Notícias sobre Vacinação:** https://www.cnnbrasil.com.br/tudo-sobre/vacina/
* **Governo do Estado de São Paulo - Notícias sobre Vacina:** https://www.saopaulo.sp.gov.br

Ao fim, excluir todas as imagens e áudios enviados para a IA

In [62]:
delete_files()

Deleting 20 images. This might take a bit...
Deleted /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:00.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/es5v6lj20pju
Deleted /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:01.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/kbzn69n01iis
Deleted /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:02.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/7i0v22mn61bz
Deleted /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:03.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/q1xewoncko2o
Deleted /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_7268a542 (1)_mp4_frame00:04.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/t8hgm6r3nxil
Deleted /content/frames/Vídeo do WhatsApp de 2024-05-09 à(s) 18_31_11_